# **Resumen automático**

Práctica de NLP para evaluar la calidad de resúmenes generados de manera automática con ROUGE score.

**Por:** Martínez Cadena Gustavo

---

## **TF-IDF**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import nltk
import re
import string
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
def cargar_texto(path: str) -> str:
  with open(path, "r") as archivo:
    texto = archivo.read()
  return texto

In [ ]:
def crear_oracion(texto:str) -> list[str]:
  texto_por_oracion = []
  oracion = ""
  for letra in texto:
    if letra == ".":
      texto_por_oracion.append(oracion.strip()) # quita espacio inicial
      oracion = ""
    else:
      oracion += letra

  return texto_por_oracion

In [ ]:
def no_caracter_especial(texto: str) -> str:
  limpiar = re.findall(r"[^-_—¡!¿?'.,;:1234567890“”\[\]\"`]", texto) # quita los caracteres especiales con expresión regular
  return ''.join(limpiar) # list[str] -> str

In [ ]:
def preprocesar(texto: str) -> list[str]:
  # normalizar
  texto = texto.lower()
  # quitar caracteres especiales
  texto = no_caracter_especial(texto)
  # tokenizar
  nltk.download('punkt_tab')
  tokens = word_tokenize(texto, language="spanish")
  # stopwords
  nltk.download("stopwords")
  stopwords_esp = set(stopwords.words("spanish"))
  no_stopwords = []

  for palabra in tokens:
    if palabra not in stopwords_esp:
      no_stopwords.append(palabra)
  #stemming
  spanish_stemmer = SnowballStemmer('spanish')
  texto_stemming = []
  for palabra in no_stopwords:
    texto_stemming.append(spanish_stemmer.stem(palabra))

  return texto_stemming

In [ ]:
def tf_idf(corpus: list[str]):
  vectorizer = TfidfVectorizer()
  X = vectorizer.fit_transform(corpus)
  return vectorizer.get_feature_names_out(), X.toarray()

In [ ]:
def resumen_tf_idf(path: str)-> str:
  # Cargar txt y dividirlo por oraciones
  texto = cargar_texto(path)
  dividir_por_oracion = crear_oracion(texto)

  # preprocesamiento de texto por oraciones
  texto_preprocesado = []
  for oracion in dividir_por_oracion:
    texto_preprocesado.append(preprocesar(oracion))

  # List[List[str]] -> list[str]
  lista_oraciones = []
  for oracion in texto_preprocesado:
    lista_oraciones.append(' '.join(oracion))

  # TF-IDF
  lista_palabras, X = tf_idf(lista_oraciones)

  # Media de score
  filas, columnas = X.shape
  score_media = []
  for i in range(filas):
    elementos_diferentes_0 = 0
    suma_score = 0
    for j in range(columnas):
      if X[i, j] != 0:
        suma_score += X[i, j]
        elementos_diferentes_0 += 1
    if elementos_diferentes_0 > 0:
      score_media.append(suma_score/elementos_diferentes_0)
    else:
      score_media.append(0)

  idx_doc_relevante = np.argsort(-np.array(score_media)) # obtener índices ordenados por relevancia
  sublista = idx_doc_relevante[:26] # Número de índices arbitrario [NOTA] crear método para calcular este número automáticamente
  sublista = np.sort(sublista) # Se ordena la sublista para evitar que se pierda el sentido del texto

  # Generar resumen
  resumen= ""
  for indice in sublista:
    resumen += dividir_por_oracion[indice] + ". "

  return resumen

## **Resumen basado en las oraciones más largas**

In [ ]:
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

def resumen_oracion_larga(path: str, n=4):
    texto = cargar_texto(path)
    # Genera un resumen automático seleccionando las n oraciones más largas.
    oraciones = sent_tokenize(texto)
    oraciones_ordenadas = sorted(oraciones, key=lambda x: len(x), reverse=True)
    resumen = ' '.join(oraciones_ordenadas[:n])
    return resumen

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## **Evaluación de resumen**

**Rouge** (Recall-Oriented Understudy for Gisting Evaluation) es una métrica para evaluar la calidad de los resúmenes generados de manera automática. Esta evaluación compara el resumen generado automáticamente con uno de referencia.


1.   **Rouge-1:** Coincidencia de palabras.
2.   **Rouge-2:** Coincidencia de pares de palabras.
3.   **Rouge-L:** Coincidencia en la secuencia de palabras más larga.



In [ ]:
from rouge_score import rouge_scorer

In [ ]:
def evaluar_resumen(resumen_generado: str, resumen_referencia: str):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(resumen_referencia, resumen_generado)
    return scores

In [ ]:
def imprimir_resultados(resultados):
  # Evaluación de resúmenes
  print("\nResultados de evaluación ROUGE:\n")
  for metrica, score in resultados.items():
      print(f"{metrica}:")
      print(f"  Precisión: {score.precision:.2f}")
      print(f"  Recall: {score.recall:.2f}")
      print(f"  F1-Score: {score.fmeasure:.2f}\n")

In [ ]:
# Main ==========================================================
# path
path_original = "/content/texto_original.txt"
path_referencia = "/content/resumen_referencia.txt"

# Obtener resumen
resultado_tf_idf = resumen_tf_idf(path_original)
resultado_oracion_larga = resumen_oracion_larga(path_original)
resumen_referencia = cargar_texto(path_referencia)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /ro

### **Impresión de resultados**

In [ ]:
resultado_oracion_larga

'Había una vez una niña muy querida por su madre y su abuela, quienes le regalaron una capa roja que le sentaba tan bien que todos la llamaban Caperucita Roja. El lobo, con gran amabilidad, le preguntó a dónde iba, y la niña, olvidando la advertencia de su madre, le contó que iba a casa de su abuela. Un día, su madre le pidió que llevara una cesta con pasteles y mantequilla a su abuela enferma, que vivía al otro lado del bosque.'

In [ ]:
resultado_tf_idf

'Antes de salir, su madre le advirtió que no hablara con extraños ni se apartara del camino. Al llegar, el lobo devoró a la abuela y se disfrazó con su ropa para engañar a la niña. Antes de que pudiera reaccionar, el lobo la devoró también. Agradecidas, llenaron el vientre del lobo con piedras pesadas, y cuando despertó, cayó muerto. Desde entonces, Caperucita Roja prometió no desobedecer las advertencias de su madre nunca más. '

In [ ]:
resumen_referencia

'Caperucita Roja, una niña querida por su familia, es enviada a casa de su abuela enferma cruzando el bosque.\nEn el camino, desobedece a su madre y habla con un lobo que, engañándola, llega primero a la casa, devora a la abuela y después a Caperucita.\nUn cazador salva a ambas cortando el vientre del lobo. La niña aprende la lección de no hablar con extraños.'

In [ ]:
resultados_1 = evaluar_resumen(resultado_tf_idf, resumen_referencia)
resultados_2 = evaluar_resumen(resultado_oracion_larga, resumen_referencia)

In [ ]:
imprimir_resultados(resultados_1)


Resultados de evaluación ROUGE:

rouge1:
  Precisión: 0.47
  Recall: 0.51
  F1-Score: 0.49

rouge2:
  Precisión: 0.19
  Recall: 0.21
  F1-Score: 0.20

rougeL:
  Precisión: 0.20
  Recall: 0.22
  F1-Score: 0.21



In [ ]:
imprimir_resultados(resultados_2)


Resultados de evaluación ROUGE:

rouge1:
  Precisión: 0.43
  Recall: 0.53
  F1-Score: 0.47

rouge2:
  Precisión: 0.17
  Recall: 0.21
  F1-Score: 0.19

rougeL:
  Precisión: 0.22
  Recall: 0.28
  F1-Score: 0.25



### **Prueba con otro texto**

**Texto original:** Breve historia de Facebook: 20 años entre la innovación y la polémica

**De:** National Geographic

**Link:** https://www.nationalgeographic.com.es/mundo-ng/breve-historia-facebook-20-anos-entre-innovacion-polemica_21537

In [ ]:
# path
path_original = "/content/Breve historia de Facebook.txt"
path_referencia = "/content/resumen_referencia_facebook.txt"

# Obtener resumen
resultado_tf_idf = resumen_tf_idf(path_original)
resultado_oracion_larga = resumen_oracion_larga(path_original)
resumen_referencia = cargar_texto(path_referencia)

resultados_1 = evaluar_resumen(resultado_tf_idf, resumen_referencia)
resultados_2 = evaluar_resumen(resultado_oracion_larga, resumen_referencia)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /ro

In [ ]:
resumen_referencia

'Facebook fue lanzado el 4 de febrero de 2004 por Mark Zuckerberg y varios compañeros durante sus años en Harvard. Inicialmente se llamó TheFacebook y rápidamente ganó popularidad entre los estudiantes universitarios. En 2005 compraron el dominio facebook.com y en 2006 la red se abrió al público general, permitiendo también el acceso a empresas para marketing y publicidad.\n\nEn 2008, Zuckerberg ya era millonario y la red social había superado a MySpace. Ese mismo año resolvió una disputa legal con antiguos compañeros, comprando su proyecto ConnectU. A lo largo del tiempo, Facebook incorporó funciones clave como el botón de "me gusta", grupos, videollamadas, y adquirió empresas como Instagram (2012) y posteriormente WhatsApp. También en 2012 salió a bolsa, recaudando 16.000 millones de dólares y alcanzando los mil millones de usuarios.\n\nLa plataforma ha enfrentado varias polémicas relacionadas con la privacidad y el uso de datos, incluyendo una sanción en 2017 por parte de la Agencia

In [ ]:
resultado_tf_idf

'Breve historia de Facebook. La plataforma fue cerrada, pero su éxito se había hecho evidente: en 4 horas acumuló más de 22. 000 visitas. Zuckerberg aprendió algo: el potencial que esconde una red social universitaria. Tuvo entonces una idea mayor. El sitio fue un éxito inmediato y en 24 horas, TheFacebook tenía ya 1. 200 usuarios registrados. Un mes después, la mitad de estudiantes de Harvard había creado sus perfiles. En los meses posteriores a su creación, TheFacebook creció rápidamente. En agosto del año 2005, la empresa compró el dominio facebook. com, que le costó 200. 000 dólares y renunció al \'The\': a partir de entonces fue solo Facebook. En septiembre, Facebook se abrió a los estudiantes de secundaria. A finales de año, la plataforma ya tenía seis millones de usuarios. En diciembre de este año, los usuarios habían aumentado hasta los doce millones. En el año 2008, Zuckerberg tenía 24 años y ya era millonario. Pero el caso que le enfrentaba con sus antiguos compañeros de clas

In [ ]:
resultado_oracion_larga

'Facebook pagó con 1,2 millones en acciones a cada uno de los tres estudiantes que habían acusado a Zuckerberg (Divya Narendra, Tyler Winklevoss y Cameron Winklevoss) y la empresa se comprometió a la compra de ConnectU, la plataforma que estaban creando los otros estudiantes en el momento en el que Zuckerberg ideó Facebook. En el año 2017, la Agencia Española de Protección de Datos impuso una sanción a Facebook de 1,2 millones de euros por vulnerar la normativa sobre protección de datos personales de los usuarios y demostró que Facebook almacenaba y usaba información de los usuarios para fines publicitarios y sin autorización previa. En la sala, estaban los familiares de niños y niñas que supuestamente habían sido perjudicados por las redes sociales: víctimas de abusos sexuales, extorsiones y daños en la salud mental -aunque sobre estos últimos, Zuckerberg defendió que no había suficiente evidencia científica que lo demostrara-. Nadie debería pasar por las cosas que sus familias han su

In [ ]:
resultados_1

{'rouge1': Score(precision=0.3843283582089552, recall=0.49047619047619045, fmeasure=0.43096234309623427),
 'rouge2': Score(precision=0.11610486891385768, recall=0.14832535885167464, fmeasure=0.13025210084033612),
 'rougeL': Score(precision=0.1791044776119403, recall=0.22857142857142856, fmeasure=0.200836820083682)}

In [ ]:
resultados_2

{'rouge1': Score(precision=0.41037735849056606, recall=0.4142857142857143, fmeasure=0.4123222748815166),
 'rouge2': Score(precision=0.0995260663507109, recall=0.10047846889952153, fmeasure=0.09999999999999999),
 'rougeL': Score(precision=0.17452830188679244, recall=0.1761904761904762, fmeasure=0.17535545023696683)}

# Observaciones

1. **TIpos de errores apreciables:**  

  -   Continuos fallos en la fluidez del resumen. Tanto el método de la oración más larga como el de TF-IDF no cerraban las ideas que desarrollaban y tenían estructuras gramaticales extrañas.
  -   Algunas de las ideas principales de los textos se perdían.



2. **Posibles mejoras:**

  - Para el método TF-IDF idealmente se debería proporcionar los conceptos clave que sean de interés.
  - En el caso de los artículos (en nuestro caso de estudio la historia de Facebook), detectar los subtemas de los que se compone el texto podría servir para extraer las ideas principales de cada parte.